The metrics functions expect `predictions` to be a `pandas.DataFrame`:

* Index: `target_datetime_utc`
* Columns:
  - `t0_datetime_utc`
  - `forecast_pv_yield`  # Values must be in the range [0, 1]

The `actual` values are a `pd.Series`, where the index is the `target_datetime_utc`,
and the values are the `actual_pv_yield` (in the range [0, 1]).

Metrics are defined by a `metrics_pipeline` which is a tuple of one or more `callable` objects.
For example:

```python
# Define re-usable metrics pipelines: The keys of the dict are human-readable metric names,
# and the values of the dict are the pipeline tuples.
metrics_pipelines: dict[str, tuple[Callable, ...]] = {
    "MAE in MW, ignoring night": (
        # Multiply the `predictions` and the `actual` by national installed PV capacity:
        Denormalize(national_pv_capacity_mwp_per_target_datetime_utc),
        # Drop any values that occur at "night". Where "night" is defined as when the Sun
        # is below a particular angle in the sky:
        IgnoreNight(latitude, longitude, threshold_for_sun_angle_in_degrees=-5),
        # Compute the absolute error for each timestep:
        absolute_error,
        # Compute the mean across all timesteps:
        mean,
    ),
    "NMAE per month, including night": (
        # Compute the absolute error for each timestep:
        absolute_error,
        # Group by month:
        GroupBy(pd.Grouper(freq="M")),
        # Compute the mean absolute error per month:
        mean,
    ),
    "NMAE per hour, ignoring night": (
        IgnoreNight(latitude, longitude, threshold_for_sun_angle_in_degrees=-5),
        absolute_error,
        # Group by hour of day:
        GroupBy(pd.Grouper(freq="H")),
        # Compute the mean absolute error per hour of the day:
        mean,
    )
}

# Run the pipelines:
metrics_results: dict[str, Union[Number, pd.Series, pd.DataFrame]] = run_all_pipelines(
    metrics_pipelines, predictions, actual)
```

In [ ]:
import pandas as pd
from typing import Callable, Union
from numbers import Number

def run_all_pipelines(
    metrics_pipelines: dict[str, tuple[Callable, ...]],
    predictions: pd.DataFrame, 
    actual: pd.Series,
) -> dict[str, Union[Number, pd.Series, pd.DataFrame]]:
    metrics_results: dict[str, Union[Number, pd.Series, pd.DataFrame]] = {}
    for metric_name, pipeline in metrics_pipelines.items():
        metrics_results[metric_name] = run_pipeline(pipeline, predictions, actual)
    return metrics_results

def run_pipeline(
    pipeline: tuple[Callable, ...], 
    predictions: pd.DataFrame, 
    actual: pd.Series
) -> Union[Number, pd.Series, pd.DataFrame]:
    # `output` starts as the predictions and actual data, and `output` will be
    # transformed by each function in the pipeline.
    output: Union[dict, Number, pd.Series, pd.DataFrame] = dict(predictions=predictions, actual=actual)
    for function in pipeline:
        if isinstance(output, dict):
            output = function(**output)
        else:
            output = function(*output)
    return output